#### Load Data


In [65]:
import numpy as np
import os
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline

In [13]:
Data_Path = os.path.join(os.getcwd(),'data','pima_indians_diabetes_data.csv')
data = np.loadtxt(Data_Path, delimiter=',')
data

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [38]:
data.shape

(768, 9)

In [50]:
X = data[:,0:8]
y = data[:,8]

In [51]:
X.shape

(768, 8)

In [52]:
y.shape

(768,)

In [75]:
X = torch.tensor(X, dtype = torch.float32)
y = torch.tensor(y, dtype = torch.float32).reshape(-1,1)

In [77]:
y.shape

torch.Size([768, 1])

#### Build the Model With Sequential


In [79]:
model = nn.Sequential(
    nn.Linear(in_features = 8, out_features = 12),
    nn.ReLU(),
    nn.Linear(in_features = 12, out_features = 8),
    nn.ReLU(),
    nn.Linear(in_features = 8, out_features = 1),
    nn.Sigmoid()
)
model

Sequential(
  (0): Linear(in_features=8, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)

#### Build the Model With OOP

In [138]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(in_features = 8, out_features = 12) # a1
        self.act1 = nn.ReLU() # z1
        self.hidden2 = nn.Linear(in_features = 12, out_features = 8) # a2
        self.act2 = nn.ReLU() #z2
        self.output = nn.Linear(in_features = 8, out_features = 1) #output
        self.act_output = nn.Sigmoid() #y-pred

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
        

In [139]:
pm_model = PimaClassifier()
pm_model

PimaClassifier(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)

#### Preparation for Training

In [153]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(params = pm_model.parameters() , lr = 0.001)

#### Training a Model

In [169]:
n_epochs = 100
batch_size = 10
                
for epoch in range(n_epochs):
    #              Start Stop    Step = 10
    for i in range( 0,   len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y__batch_pred = pm_model(Xbatch)
        y__batch_true = y[i:i+batch_size]
        #            batch preds ( Predicted )      trye values ( target )
        batch_loss = loss_fn(y__batch_pred, y__batch_true)
        # Resets the gradients of all optimized for each batch
        optimizer.zero_grad()
        #applying fine tuning on weight with backward()
        batch_loss.backward()
        # Performs a single optimization step.
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {batch_loss}')
    
# compute accuracy
y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")
 
# make class predictions with the model
predictions = (model(X) > 0.5).int()
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

Finished epoch 0, latest loss 0.24020284414291382
Finished epoch 1, latest loss 0.21486763656139374
Finished epoch 2, latest loss 0.2384396493434906
Finished epoch 3, latest loss 0.23898053169250488
Finished epoch 4, latest loss 0.22514288127422333
Finished epoch 5, latest loss 0.22914761304855347
Finished epoch 6, latest loss 0.23141615092754364
Finished epoch 7, latest loss 0.22154535353183746
Finished epoch 8, latest loss 0.21900150179862976
Finished epoch 9, latest loss 0.22378407418727875
Finished epoch 10, latest loss 0.21187910437583923
Finished epoch 11, latest loss 0.2246689349412918
Finished epoch 12, latest loss 0.26140525937080383
Finished epoch 13, latest loss 0.21472758054733276
Finished epoch 14, latest loss 0.230550155043602
Finished epoch 15, latest loss 0.2400076687335968
Finished epoch 16, latest loss 0.22704815864562988
Finished epoch 17, latest loss 0.25993967056274414
Finished epoch 18, latest loss 0.2258155792951584
Finished epoch 19, latest loss 0.22875748574733

#### Make Predictions

In [168]:
# make probability predictions with the model
predictions = (model(X) > 0.5).int()

In [166]:
predictions[:5]

tensor([[0.1703],
        [0.1660],
        [0.1673],
        [0.0314],
        [0.0051]], grad_fn=<SliceBackward0>)